In [1]:
import src.environ

/home/mengkjin/Workspace/learndl
Use device name: NVIDIA GeForce RTX 4090


FileNotFoundError: [Errno 2] No such file or directory: '/home/mengkjin/Workspace/learndl/../configs/config_logger.yaml'

In [1]:
from run_model import predict_new


24-03-25 16:38:12|MOD:data_preprocessing|: if_train is False , Data Processing start!
24-03-25 16:38:12|MOD:data_preprocessing|: 3 datas :['y', 'trade_day', 'trade_30m']


Mon Mar 25 16:38:12 2024 : y start ...
labels blocks reading 10days/lag1 Data1D's ...... cost 0.26 secs
labels blocks reading 20days/lag1 Data1D's ...... cost 0.20 secs
labels blocks merging ...... cost 0.05 secs
models blocks reading risk_model/exposure Data1D's ...... cost 1.43 secs
models blocks merging ...... cost 0.00 secs
y blocks process ...... cost 2.27 secs
y blocks masking ...... cost 0.11 secs
y blocks saving  ...... cost 0.09 secs
y blocks norming ...... cost 0.00 secs
Mon Mar 25 16:38:17 2024 : y finished! Cost 4.80 Seconds
Mon Mar 25 16:38:17 2024 : trade_day start ...
trade_day blocks reading day/trade Data1D's ...... cost 0.44 secs
trade_day blocks merging ...... cost 0.00 secs
trade_day blocks process ...... cost 0.13 secs
trade_day blocks masking ...... cost 0.09 secs
trade_day blocks saving  ...... cost 0.10 secs
trade_day blocks norming ...... cost 0.00 secs
Mon Mar 25 16:38:18 2024 : trade_day finished! Cost 0.77 Seconds
Mon Mar 25 16:38:18 2024 : trade_30m start .

24-03-25 16:38:42|MOD:data_preprocessing|: Data Processing Finished! Cost 30.18 Seconds


... cost 0.83 secs
trade_30m blocks norming ...... cost 0.00 secs
Mon Mar 25 16:38:42 2024 : trade_30m finished! Cost 24.35 Seconds
Load  2 DataBlocks ...... cost 0.06 secs
Align 2 DataBlocks ...... cost 0.12 secs


In [9]:
predict_new(model_path = './model/gru_day' , model_num = 0)

Load  2 DataBlocks ...... cost 0.05 secs
Align 2 DataBlocks ...... cost 0.10 secs
